In [1]:
import investpy as inv
from datetime import datetime
import pandas as pd
import numpy as np

today = datetime.today().strftime("%d/%m/%Y")

## Functions

In [10]:
def get_assets_data_frames(assets: list, asset_function: list, country: str, start_date: str, end_date: str) -> list:
    """
    Get asset OHLCV values from investpy based on one country.

    Args: 
    - assets (list): assets to be downloaded
    - asset_function (list): investpy function that get historical data
    - country (str): origin country of the assets
    - start_date (str): initial date of the historical data
    - end_date (str): end date of the historical data

    Returns:
    - data_frame (list<pd.DataFrame>): assets data frames
    
    """

    data_frames = []

    for asset in assets:

        data_frame = asset_function(asset,
                                    country=country,
                                    from_date=start_date,
                                    to_date=end_date)

        data_frames.append(data_frame)

    return data_frames

In [11]:
def build_multi_index_tuples(header: list, sub_header: list) -> list:
    """
    Build list of tuples that construct a multiheader data frame.

    Args:
    - header (list): first header columns
    - sub_header (list): second header columns

    Returns:
    - tuples (list): multiheader pairs

    >>> build_multi_index_tuples(['Open', 'Close'], ['BPAC11', 'OIBR3', 'PETR4', 'MGLU3'])
    >>> [('Open', 'BPAC11'),
         ('Open', 'OIBR3'),
         ('Open', 'PETR4'),
         ('Open', 'MGLU3'),
         ('Close', 'BPAC11'),
         ('Close', 'OIBR3'),
         ('Close', 'PETR4'),
         ('Close', 'MGLU3')]

    """

    tuples = []

    for head in header:
        for sub_head in sub_header:
            tuples.append((head, sub_head))

    return tuples

In [12]:
def build_multi_index_data_frame(data_frames: list, sub_header: list, header_columns: list) -> pd.DataFrame:
    """
    Build a multiheader data frame. With Colums as header and assets as sub header.

    Args:
    - data_frames (list): list of data frames.
    - sub_header (list): sub header columns.
    - header_columns (list): header columns.
    """

    tuples = build_multi_index_tuples(header_columns, sub_header)

    multi_header = pd.MultiIndex.from_tuples(tuples)

    df = pd.concat(data_frames, axis=1).loc[:, dict(tuples).keys()]

    df.columns = multi_header

    return df

In [15]:
def get_portfolio_prices(stocks: list, funds: list, etfs: list, start_date: str, end_date=today) -> pd.DataFrame:
    """
    Get multiheader asset prices data frame. OHLC as principal header and asset names as sub header.

    Args: 
    - stocks (list): stock names
    - funds (list): funds names
    - etfs (str): etfs names
    - start_date (str): initial date of the historical data
    - end_date (str): end date of the historical data

    Returns:
    - data_frame (list<pd.DataFrame>): assets data frames
    
    """
    data_frames_stocks = get_assets_data_frames(
        stocks, inv.get_stock_historical_data, 'brazil', start_date=start_date, end_date=end_date)
    data_frames_funds = get_assets_data_frames(
        funds, inv.get_fund_historical_data, 'brazil', start_date=start_date, end_date=end_date)
    data_frames_etfs = get_assets_data_frames(
        etfs, inv.get_etf_historical_data, 'brazil', start_date=start_date, end_date=end_date)

    data_frames = [*data_frames_stocks, *data_frames_funds, *data_frames_etfs]

    assets = [*stocks, *funds, *etfs]

    portfolio_prices = build_multi_index_data_frame(
        data_frames, assets, ['Close', 'Open', 'High', 'Low'])

    return portfolio_prices

## Tests

In [16]:
stocks = ['BPAC11', 'OIBR3', 'PETR4', 'MGLU3']
funds = ['Arx Income Fundo De Investimento Em Acoes', 'Visia Zarathustra Fundo De Investimento Em Cotas De Fundo De Investimento Multimercado']
etfs = ['Fundo de Invest Ishares SP 500']

columns = ['Open', 'High', 'Low', 'Close']

In [17]:
build_multi_index_tuples(['Open', 'Close'], ['BPAC11', 'OIBR3', 'PETR4', 'MGLU3'])

[('Open', 'BPAC11'),
 ('Open', 'OIBR3'),
 ('Open', 'PETR4'),
 ('Open', 'MGLU3'),
 ('Close', 'BPAC11'),
 ('Close', 'OIBR3'),
 ('Close', 'PETR4'),
 ('Close', 'MGLU3')]

In [18]:
get_portfolio_prices(stocks, funds, etfs, '01/01/2015')

Close                      \
            BPAC11 OIBR3  PETR4  MGLU3   
Date                                     
2015-01-02     NaN  9.12   8.98   0.22   
2015-01-05     NaN  8.40   8.26   0.22   
2015-01-06     NaN  7.10   7.99   0.22   
2015-01-07     NaN  7.08   8.32   0.23   
2015-01-08     NaN  7.05   8.81   0.23   
...            ...   ...    ...    ...   
2021-04-13   98.60  1.81  23.97  22.20   
2021-04-14   99.50  1.75  24.35  22.12   
2021-04-15   99.23  1.73  23.09  22.20   
2021-04-16  101.14  1.70  22.95  21.92   
2021-04-19   99.75  1.68  24.28  21.65   

                                                      \
           Arx Income Fundo De Investimento Em Acoes   
Date                                                   
2015-01-02                                       NaN   
2015-01-05                                       NaN   
2015-01-06                                       NaN   
2015-01-07                                       NaN   
2015-01-08                                       NaN   
...                                              ...   
2021-04-13                                    74.824   
2021-04-14                                       NaN   
2021-04-15                                    75.356   
2021-04-16                                       NaN   
2021-04-19                                       NaN   

                                                                                                   \
           Visia Zarathustra Fundo De Investimento Em Cotas De Fundo De Investimento Multimercado   
Date                                                                                                
2015-01-02                                                NaN                                       
2015-01-05                                                NaN                                       
2015-01-06                                                NaN                                       
2015-01-07                                                NaN                                       
2015-01-08                                                NaN                                       
...                                                       ...                                       
2021-04-13                                              3.848                                       
2021-04-14                                                NaN                                       
2021-04-15                                                NaN                                       
2021-04-16                                                NaN                                       
2021-04-19                                                NaN                                       

                                             Open               ...  \
           Fundo de Invest Ishares SP 500  BPAC11 OIBR3  PETR4  ...   
Date                                                            ...   
2015-01-02                          55.80     NaN  9.40   9.58  ...   
2015-01-05                          55.75     NaN  9.12   8.77  ...   
2015-01-06                          54.85     NaN  8.36   8.36  ...   
2015-01-07                          55.17     NaN  7.14   8.21  ...   
2015-01-08                          55.80     NaN  7.08   8.43  ...   
...                                   ...     ...   ...    ...  ...   
2021-04-13                         257.20   99.03  1.89  23.98  ...   
2021-04-14                         254.29   98.83  1.81  24.20  ...   
2021-04-15                         254.95   99.65  1.77  23.70  ...   
2021-04-16                         255.00   99.77  1.72  23.13  ...   
2021-04-19                         251.65  100.50  1.71  22.95  ...   

                                                High  \
           Arx Income Fundo De Investimento Em Acoes   
Date                                                   
2015-01-02                                       NaN   
2015-